In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sqlalchemy import create_engine

In [2]:
widths = (8,7,4,13,43)
header_pd = pd.read_fwf('Labeled_data.txt', widths = widths,skiprows=7, nrows=27)
labeled_data = pd.read_csv('Labeled_data.txt', header=None, delim_whitespace=True, skiprows=37) # extract data
labeled_data.columns = header_pd.iloc[:,3]
labeled_data.head()
#shuffle data
labeled_data_sampled = labeled_data.sample(n=5, random_state=1)

In [3]:
engine = create_engine('sqlite:///save_LC.db', echo=False)
sqlite_connection = engine.connect()

circle_radius = 0.00028 # 1 arcsec = 0.00028 degress
t_format = "ipac_table"
table_format = "FORMAT=" + str(t_format)
flag_mask = 32768
mask = "BAD_CATFLAGS_MASK=" + str(flag_mask)
collect="COLLECTION="+"ztf_dr2"
numobs = "NOBS_MIN=20"
filter_band = "g"
label = []
SourceID =[]
start_time = time.time()
k = 0
for i in labeled_data_sampled.index:
    label.append(labeled_data_sampled.Type[i])
    SourceID.append(labeled_data_sampled.SourceID[i])
    ra = labeled_data_sampled.RAdeg[i]
    dec = labeled_data_sampled.DEdeg[i]
    circle = "POS=CIRCLE"+"+"+str(ra)+"+"+str(dec)+"+"+str(circle_radius)
    band = "BANDNAME="+ filter_band
    params = circle + "&" +  mask + "&" + numobs + "&" + collect + "&" + table_format
    
    url= "https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?" + params
    data = pd.read_csv(url, header=None, delim_whitespace=True, skiprows=55) # extract data
    header = pd.read_csv(url, header=None, sep='|', skiprows=50,usecols=range(1,25), nrows=1)
    data.columns = header.iloc[0].str.strip()
    lc = data
    sqlite_table = labeled_data_sampled.ID[i]
    lc.to_sql(sqlite_table, sqlite_connection, if_exists='replace')
    print(k)
    k = k+1
end_time = time.time()# total time taken
print(f"Runtime of the program is {end_time} - {start_time}")

KeyboardInterrupt: 

In [ ]:
labeled_data.info()

In [ ]:
print(labeled_data_sampled.ID[i])

In [ ]:
df = pd.read_sql_query('SELECT * FROM "ZTFJ185629.89-050650.2" LIMIT 3',sqlite_connection)
df.head()

In [ ]:
engine = create_engine('sqlite:///save_LC.db', echo=False)
sqlite_connection = engine.connect()

In [5]:
from sqlalchemy import MetaData
from sqlalchemy import Table

In [6]:
metadata = MetaData()
print(metadata.tables)

FacadeDict({})


In [ ]:
LC_data = pd.read_csv('http://variables.cn:88/seldataz.php?SourceID=781604', header=None,skiprows=0) # extract data
header = pd.read_csv('http://variables.cn:88/seldataz.php?SourceID=781604', header=None,skiprows=0) # extract data
LC_data.columns = header.iloc[0].str.strip()

In [ ]:
LC_data.head()

In [ ]:
header.head()